In [1]:
import warnings
warnings.filterwarnings('ignore')

from datasets import Dataset
import pandas as pd
# import numpy as np

# from tqdm import tqdm
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
# # from transformers import GPT2LMHeadModel, GPT2TokenizerFast # For GPT-like models

# # MODEL_CHECKPOINT = "Helsinki-NLP/opus-mt-en-hi"
# MODEL_CHECKPOINT = "google/flan-t5-small"
# # MODEL_CHECKPOINT = "bert-base-multilingual-uncased"

# # tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, strip_accents=True)
# model = AutoModel.from_pretrained(MODEL_CHECKPOINT)

# MODEL_CHECKPOINT = "gpt2"

# tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_CHECKPOINT)
# # model = GPT2LMHeadModel.from_pretrained(MODEL_CHECKPOINT)

# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# MODEL_CHECKPOINT ='google/flan-t5-small'
MODEL_CHECKPOINT ="google/mt5-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
encodes = tokenizer.encode('दर्जनों सीढ़ियाँ चढ़ने के बाद मेरी साँसें फूल जाती थीं लेकिन अब जब मैं नीचे बैठता हूँ तो भी मुझे साँस लेने में कठिनाई होती है।')
outputs = tokenizer.decode(encodes)
outputs
# encodes

'दर्जनों सीढ़ियाँ चढ़ने के बाद मेरी साँसें फूल जाती थीं लेकिन अब जब मैं नीचे बैठता हूँ तो भी मुझे साँस लेने में कठिनाई होती है।</s>'

In [5]:
df = pd.read_csv('train.csv')
df.dropna(inplace=True)
df.head()

,file,og_transcription,ms_asr_transcription
0,../data/audios/hindi/train/hin_audio_0.wav,जब भी मैं उसे याद करता हूं तो मैं उदास हो जाता...,जब भी मैं उसे याद करता हूँ।
1,../data/audios/hindi/train/hin_audio_1.wav,जब मैं भारी चीजें उठाता हूं तो मुझे ऐसा लगता ह...,जब मैं भारी चीजें उठाता हूँ तो मुझे ऐसा लगता ह...
2,../data/audios/hindi/train/hin_audio_2.wav,जब मैं अपना हाथ हिलाता हूं तो बहुत अधिक दर्द ह...,जब मैं अपना हाथ हिलाता हूँ तो बहुत अधिक दर्द ह...
3,../data/audios/hindi/train/hin_audio_3.wav,मेरे बेटे के होंठ में छेद हो गया था और वह सूज ...,मेरे बेटे के होम्स में छेद हो गया था और वो सूझ...
4,../data/audios/hindi/train/hin_audio_4.wav,मेरी पीठ के निचले हिस्से की मांसपेशियाँ दर्द क...,मेरी पीठ के निचले हिस्से की मांसपेशियां दर्द क...


In [6]:
SOURCE = 'ms_asr_transcription'

TARGET = 'og_transcription'

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df[SOURCE], df[TARGET], test_size=0.10,
                                                    shuffle=True,
                                                    random_state=100)

In [8]:
def prep_data_for_model_fine_tuning(source_lang: list, target_lang: list) -> list:
    data_dict = dict()
    data_dict['data'] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict[SOURCE] = sr_text
        temp_dict[TARGET] = tr_text

        data_dict['data'].append(temp_dict)

    return data_dict

In [9]:
training_data = prep_data_for_model_fine_tuning(x_train.values, y_train.values)
testing_data = prep_data_for_model_fine_tuning(x_test.values, y_test.values)

In [10]:
training_data['data'][:5]

[{'ms_asr_transcription': 'जब मैं महानसों को छूता हूँ तो मुझे गहरी झुनझुनी महसूस होती है और उसके बाद बुरी अनुती होती है।',
  'og_transcription': 'जब मैं मुहांसों को छूता हूं तो मुझे गहरी झुनझुनी महसूस होती है और उसके बाद बुरी अनुभूति होती है'},
 {'ms_asr_transcription': 'जब भी मैं अपने बालों में कम ही करता हूँ तो मेरी कम ही में बहुत सारे बाल होते हैं।',
  'og_transcription': 'जब भी मैं अपने बालों में कंघी करता हूं तो मेरी कंघी में बहुत सारे बाल होते हैं।'},
 {'ms_asr_transcription': 'जब मैं अचानक हरकत करता हूँ तो मुझे चक्कर आने लगता है।',
  'og_transcription': 'जब मैं अचानक हरकत करता हूं तो मुझे चक्कर आने लगता है'},
 {'ms_asr_transcription': 'मेरी पीक में समस्या है, मैं इसे बढ़ा नहीं सकता था।',
  'og_transcription': 'मेरी पीठ में समस्या है मैं इसे बढ़ा नहीं सकता'},
 {'ms_asr_transcription': 'मैं इस भयानक एहसास को अपने कंधे पर रख कर बर्दाश्त नहीं कर सकता।',
  'og_transcription': 'मैं इस भयानक एहसास को अपने कंधे पर रखकर बर्दाश्त नहीं कर सकता'}]

In [11]:
MAX_INPUT_LENGTH = 128

def generate_model_ready_dataset(dataset: list, source: str, target: str, tokenizer: AutoTokenizer):
    preped_data = []

    for row in dataset:
        inputs = row[source]
        targets = row[target]

        model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)#, padding=True)
        model_inputs['data'] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)
            model_inputs['labels'] = labels['input_ids']

        preped_data.append(model_inputs)

    return preped_data

In [12]:
train_data = generate_model_ready_dataset(dataset=training_data['data'],
                                        tokenizer=tokenizer,
                                        source=SOURCE,
                                        target=TARGET)

test_data = generate_model_ready_dataset(dataset=testing_data['data'],
                                        tokenizer=tokenizer,
                                        source=SOURCE,
                                        target=TARGET)

In [13]:
train_df = pd.DataFrame.from_records(train_data)
test_df = pd.DataFrame.from_records(test_data)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   attention_mask  1881 non-null   object
 1   data            1881 non-null   object
 2   input_ids       1881 non-null   object
 3   labels          1881 non-null   object
dtypes: object(4)
memory usage: 58.9+ KB


In [14]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset

Dataset({
    features: ['attention_mask', 'data', 'input_ids', 'labels'],
    num_rows: 1881
})

In [15]:
MODEL_INTENT = 'asr_correction'

original_model = MODEL_CHECKPOINT.split('/')[-1]
lang = 'hindi'
trained_model_name = f'{original_model}-finetuned-{MODEL_INTENT}-{lang}'
batch_size = 8
model_args = Seq2SeqTrainingArguments(
    trained_model_name,
    # evaluation_strategy='epoch',
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

cuda:0


In [17]:
trainer = Seq2SeqTrainer(
    model.to(device),
    model_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

  0%|          | 0/236 [00:00<?, ?it/s]

{'train_runtime': 3045.1724, 'train_samples_per_second': 0.618, 'train_steps_per_second': 0.077, 'train_loss': 2.243828466383077, 'epoch': 1.0}


TrainOutput(global_step=236, training_loss=2.243828466383077, metrics={'train_runtime': 3045.1724, 'train_samples_per_second': 0.618, 'train_steps_per_second': 0.077, 'total_flos': 203896389565440.0, 'train_loss': 2.243828466383077, 'epoch': 1.0})

In [23]:
trainer.save_model(f"{MODEL_CHECKPOINT}-finetuned-{MODEL_INTENT}-{lang}")

In [19]:
def correct(text, model=model, tokenizer=tokenizer):
    device = 'cpu'
    model.to(device)
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]
    output = model.generate(input_ids)
    translated_text = tokenizer.batch_decode(output, skip_special_tokens=False)[0]
    return translated_text

In [20]:
correct('मुझमें वो काम करने की ऊर्जा नहीं है जो मैं पहले करता था।')

'<pad>  वह काम करने की ऊर्जा नहीं है जो मैं पहले कर'

In [21]:
text = 'मुझमें वो काम करने की ऊर्जा नहीं है जो मैं पहले करता था।'
encodings = tokenizer(text, return_tensors='pt')['input_ids']
print(encodings)

decodings = tokenizer.batch_decode(encodings)
decodings

tensor([[ 6742,  6664, 10936,   259, 23978,  8520,  1100,   757,   975,   259,
         12136,   996, 78773,  3535,   830,  5903,  9422,  2917,  4510,  1048,
          1100,  1114,  3365,   378,     1]])


['मुझमें वो काम करने की ऊर्जा नहीं है जो मैं पहले करता था।</s>']

In [22]:
# MODEL_CHECKPOINT = "monsoon-nlp/hindi-bert"
# MODEL_CHECKPOINT = "gpt2"